## APIのビルドと初期化

In [1]:
from googleapiclient.discovery import build

# 利用するAPIサービス
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

# APIキー
YOUTUBE_API_KEY = 'your-api-key-here'

# API のビルドと初期化
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=YOUTUBE_API_KEY)

## 動画情報一覧を取得

In [4]:
from datetime import datetime, timedelta, timezone
import pandas as pd

        
def get_video_list_in_channel(youtube, channel_id, max_req_cnt=2):
    '''特定のチャンネルの動画情報を取得し、必要な動画情報を返す
    
        公開時刻が新しい順に50ずつリクエスト
        デフォルトでは最大2リクエストで終了
    '''

    n_requested = 50

    earliest_publishedtime =\
        datetime.now(tz=timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    req_cnt = 0
    result = []
    while True:
        response = youtube.search().list(part='snippet',
                                        channelId=channel_id,
                                        order='date',
                                        type='video',
                                        publishedBefore=earliest_publishedtime,
                                        maxResults=n_requested).execute()
        req_cnt += 1
        video_info = fetch_video_info(response)
        result.append(video_info)
        
        # 取得動画の最も遅い公開日時の1秒前以前を次の動画一覧の取得条件とする
        last_publishedtime = video_info['publishTime'].min()
        last_publishedtime_next =\
            datetime.strptime(last_publishedtime, '%Y-%m-%dT%H:%M:%SZ') - timedelta(seconds=1)
        
        earliest_publishedtime = last_publishedtime_next.strftime('%Y-%m-%dT%H:%M:%SZ')

        if req_cnt > max_req_cnt:
            # リクエスト回数がmax_req_cntを超えたらループを抜ける
            print('Result count exceeded max count {}.'.format(max_req_cnt))
            break

        if len(response['items']) < n_requested:
            # リクエストした動画数より少ない数が返った場合はループを抜ける
            print('Number of results are less than {}.'.format(n_requested))
            break

    if len(result) > 1:
        df_video_list = pd.concat(result, axis=0).reset_index(drop=True)
    else:
        df_video_list = result[0]

    return df_video_list


def fetch_video_info(response, as_df=True):
    '''APIのレスポンスから必要な動画情報を抜き出す'''
    info_list = []
    for item in response['items']:
        info = {}
        info['title'] = item['snippet']['title']
        info['kind'] = item['id']['kind']
        info['videoId'] = item['id']['videoId']
        info['description'] = item['snippet']['description']
        info['publishTime'] = item['snippet']['publishTime']
        info['channelTitle'] = item['snippet']['channelTitle']
        info['thumbnails_url'] = item['snippet']['thumbnails']['default']['url']
        info_list.append(info)
    if as_df:
        return pd.DataFrame(info_list)
    else:
        info_list


In [5]:
# B-life channel ID
channel_id = 'UCd0pUnH7i5CM-Y8xRe7cZVg'

df_video_list = get_video_list_in_channel(youtube, channel_id)

Result count exceeded max count 2.


## 2. 動画の再生時間を取得し、フィルタをかける

In [6]:
import re
import numpy as np

def get_contents_detail_core(youtube, videoids):
    '''動画の詳細情報を取得'''
    part = ['snippet', 'contentDetails']
    response = youtube.videos().list(part=part, id=videoids).execute()
    results = []
    for item in response['items']:
        info = get_basicinfo(item)
        info['duration'] = get_duration(item)
        results.append(info)
    return pd.DataFrame(results)


def get_contents_detail(youtube, videoids):
    '''必要に応じて50件ずつにIDを分割し、詳細情報を取得'''
    n_req_pre_once = 50
    
    # IDの数が多い場合は50件ずつ動画IDのリストを作成
    if len(videoids) > n_req_pre_once:
        videoids_list = np.array_split(videoids, len(videoids) // n_req_pre_once + 1)
    else:
        videoids_list = [videoids,]

    # 50件ずつ動画IDのリストを渡し、動画の詳細情報を取得
    details_list = []
    for vids in videoids_list:
        df_video_details_part = get_contents_detail_core(youtube, vids.tolist())
        details_list.append(df_video_details_part)

    df_video_details = pd.concat(details_list, axis=0).reset_index(drop=True)
    return df_video_details


def get_duration(item):
    '''動画時間を抜き出す（ISO表記を秒に変換）'''
    content_details = item['contentDetails']
    pt_time = content_details['duration']
    return pt2sec(pt_time)


def get_basicinfo(item):
    '''動画の基本情報の抜き出し'''
    basicinfo = dict(id=item['id'])
    # snippets
    keys = ('title', 'description', 'channelTitle')
    snippets = {k: item['snippet'][k] for k in keys}
    basicinfo.update(snippets)
    return basicinfo


def pt2sec(pt_time):
    '''ISO表記の動画時間を秒に変換 '''
    pttn_time = re.compile(r'PT(\d+H)?(\d+M)?(\d+S)?')
    keys = ['hours', 'minutes', 'seconds']
    m = pttn_time.search(pt_time)
    if m:
        kwargs = {k: 0 if v is None else int(v[:-1])
                    for k, v in zip(keys, m.groups())}
        return timedelta(**kwargs).total_seconds()
    else:
        msg = '{} is not valid ISO time format.'.format(pt_time)
        raise ValueError(msg)

In [7]:
# 動画ID
videoids = df_video_list['videoId'].values

# 動画の詳細情報を取得
df_video_details = get_contents_detail(youtube, videoids)


In [8]:
df_video_details.head()

,id,title,description,channelTitle,duration
0,CeyKQAFoTsI,【魔法の呼吸法】 ストレス知らずの疲れない身体をつくるヨガ☆ #395,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,3950.0
1,lJlbbbk3H_U,【ゆるヨガ】 柔軟性ゼロから始める！ 体が硬い人のためのヨガ☆,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,1117.0
2,El-hcEUklR4,【毎日15分】 全身がほぐれるフローヨガ☆ 朝ヨガにもオススメ！ #393,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,951.0
3,9IzJohZneUQ,【滝汗HIIT】 全身の脂肪燃焼に効果的☆ 代謝＆免疫力アップ！#392,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,3693.0
4,cdQVhWP-ZXE,【ぐっすり安眠】 夜寝る前のヨガで自律神経を整えよう！ 初心者にもオススメ☆ #391,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,1017.0


In [9]:
# 再生時間が10分以上、15分以下の動画に絞り込む
lower_duration = 10 * 60  # 10分以上
upper_duration = 15 * 60  # 15分以下
is_match_cond = df_video_details['duration'].between(lower_duration, upper_duration)

df_video_playlist = df_video_details.loc[is_match_cond, :]

# 出力
df_video_playlist.head()

,id,title,description,channelTitle,duration
14,nZSe3ZZUSJw,【ベッドで朝ヨガ】 起きてすぐ！全身を心地よくほぐし１日を快適に☆ #381,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,867.0
32,EYqPH91q5nE,【毎日10分】 カラダが変わる！ ヨガで自律神経をセルフケア☆ #363,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\n購入はこちら...,B-life,692.0
35,Zw_osQctQKs,骨盤のゆがみを整え、腰痛を緩和するヨガ☆ 初心者にオススメ！ #360,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\nご予約はこち...,B-life,771.0
45,kBLCmz8pODg,【1日5分で美しく引き締まる】魔法の美尻トレーニング！ ＃350,,B-life,690.0
46,IBt5l9Um_rY,【1日5分で美しく引き締まる】魔法の背中＆二の腕トレーニング！ ＃349,B-life書籍最新刊『魔法のピラティス』がいよいよ6月1日に発売決定！\n\nご予約はこち...,B-life,640.0


In [10]:
df_video_playlist['id'].values

array(['nZSe3ZZUSJw', 'EYqPH91q5nE', 'Zw_osQctQKs', 'kBLCmz8pODg',
       'IBt5l9Um_rY', 'WWxKW8ncPe0', 'CmWJJWKLeKg', 'e9HR7-3I3MA',
       'dsHgUncnWTk', 'J0B27XRg9Ck', '2S8fSj5Ox6o', 'zv0mrCF3qJc',
       '3DoZVZv2ioA', '4eD2IzDnyVc', 's7w6jLynl6c', 'd8h1oTP_0Gw',
       'GamX-b6DASY', 'V3_ICKUlwl8', 'PNYyX1vSO98', 'WL-yCF3f6OQ',
       'LTYhuOj0zfs', 'vSAtEErqDWY', 'F_vZ29OOa2M', 'MDMbpikr8oo',
       'WrHXfbKpbKM', 'l_NmT5eJjjw', 'z_JVHxVT2W0', 'aETSHfGP1jo',
       'FESpC1qN6-Y', 'U9mZS_GRmxQ', '1MC0xl3y4yU', 'QcqWYaMNvGY',
       '-1M4wiFSaDQ'], dtype=object)